In [96]:
import sqlite3
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

In [97]:
# Create a SQL connection to our SQLite database
con = sqlite3.connect("../FPA_FOD_20210617.sqlite")

cur = con.cursor()

In [98]:
cur.execute('SELECT region_id, DISCOVERY_DATE, FPA_ID FROM Fires limit 10')
cur.fetchall()

[(109, '2/2/2005 0:00', 'FS-1418826'),
 (109, '5/12/2004 0:00', 'FS-1418827'),
 (109, '5/31/2004 0:00', 'FS-1418835'),
 (109, '6/28/2004 0:00', 'FS-1418845'),
 (109, '6/28/2004 0:00', 'FS-1418847'),
 (109, '6/30/2004 0:00', 'FS-1418849'),
 (109, '7/1/2004 0:00', 'FS-1418851'),
 (109, '3/8/2005 0:00', 'FS-1418854'),
 (109, '3/15/2005 0:00', 'FS-1418856'),
 (109, '7/1/2004 0:00', 'FS-1418859')]

In [ ]:
df = pd.read_sql_query("SELECT region_id, DISCOVERY_DATE FROM Fires", con).dropna()

In [ ]:
df.head(10)

In [ ]:
df['DISCOVERY_DATE'] = pd.to_datetime(df['DISCOVERY_DATE'])
df.head()

In [ ]:
per = df.DISCOVERY_DATE.dt.to_period("M")
fires_refactor = df.groupby([per,'region_id']).count()
fires_refactor.head()

In [ ]:
fires_refactor.rename(columns={'DISCOVERY_DATE':'count'},inplace=True)
fires_refactor = fires_refactor.reset_index()
fires_refactor.head()

In [ ]:
fires_refactor['month'] = fires_refactor.iloc[:,0].dt.strftime('%m')
fires_refactor.rename(columns={'DISCOVERY_DATE':'date'},inplace=True)
fires_refactor.head()

In [ ]:
weather = pd.read_sql_query('select date, region_id, departure_avg_temp, departure_precipitation, departure_max_temp, departure_min_temp from weather',con).dropna()
weather['date'] = pd.to_datetime(weather['date']).dt.to_period("M")
weather.shape

In [ ]:
data = weather.merge(fires_refactor,how="inner",on=["date","region_id"])
data.shape

In [ ]:
data.tail(20)

In [ ]:
def build_and_compile_model(norm):
  model = keras.Sequential([
      norm,
      layers.Dense(64, activation='relu'),
      layers.Dense(64, activation='relu'),
      layers.Dense(1)
  ])

  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))
  return model

In [ ]:
data = data.astype({'region_id':float,'count':float,'month':float})
data.dtypes


In [ ]:
train_features = data.iloc[:,1:8].sample(frac=0.8, random_state=0)
test_features = data.iloc[:,1:8].drop(northeast_dataset.index)

train_labels = train_features.pop('count')
test_labels = test_features.pop('count')
train_features.head()

In [ ]:
normalizer = preprocessing.Normalization(axis=-1)
normalizer.adapt(np.array(train_features))

In [ ]:
dnn_model = build_and_compile_model(normalizer)
dnn_model.summary()

In [ ]:
%%time
history = dnn_model.fit(
    train_features, train_labels,
    validation_split=0.2,
    verbose=0, epochs=100)

In [ ]:
test_predictions = dnn_model.predict(test_features).flatten()

In [ ]:

plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values [Fire Count]')
plt.ylabel('Predictions [Fire Count]')

lims = [0, 3000]
plt.xlim(lims)
plt.ylim(lims)


In [ ]:
dnn_model.get_weights()

In [ ]:
con.close()